In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [6]:
train = pd.read_csv("data/t_train.csv")
test = pd.read_csv("data/t_test.csv")

In [23]:
y = train["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

In [31]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X)
print("Accuracy:",metrics.accuracy_score(y, predictions))

Accuracy: 0.8117977528089888


In [32]:
cats = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

def getCombinations(seq):
    combinations = list()
    for i in range(0,len(seq)):
        for j in range(i+1,len(seq)):
            combinations.append([seq[i],seq[j]])
    return combinations

res = getCombinations(cats)

In [33]:
def subsets(numbers):
    if numbers == []:
        return [[]]
    x = subsets(numbers[1:])
    return x + [[numbers[0]] + y for y in x]
 
# wrapper function
def subsets_of_given_size(numbers, n):
    return [x for x in subsets(numbers) if len(x)==n]

In [36]:
combo = subsets(cats)
combo.pop(0)

[]

In [ ]:
m = 0
res = [0, 0]
cats = []
for feature in combo:
    Y = train["Survived"]
    X = pd.get_dummies(train[feature])
    for i in range(1, 150):
        for j in range(1, 11):
            model = RandomForestClassifier(n_estimators=i, max_depth=j, random_state=1)
            x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
            model.fit(x_train, y_train)
            predictions = model.predict(x_test)
            acc = metrics.accuracy_score(y_test, predictions)
            if acc > m:
                m = acc
                res = [i, j]
                cats = feature
print(res, m, cats)

In [ ]:
Y = train['Survived']
err = 100
for features in combo:
    X = train[features]
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
    clf_ = SGDRegressor(max_iter=n_iter)
    clf_.fit(x_train, y_train)
    y_pred_sksgd=clf_.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_sksgd)
    
    if err > mse:
        err = mse
        feature = features
print(err, features)

In [ ]:
acc = 0
for i in range(len(combo)):
    Y = train['Survived']
    features = combo[i]
    X = train[features]
    classifier = SVC(kernel='rbf', random_state = 1)
    classifier.fit(x_train,y_train)
    Y_pred = classifier.predict(x_test)
    cm = confusion_matrix(y_test,Y_pred)
    accuracy = float(cm.diagonal().sum())/len(y_test)
    
    if accuracy > acc:
        acc = accuracy
        feat = features
        
print(acc, features)

In [ ]:
acc = 0
for i in range(len(combo)):
    Y = train['Survived']
    features = combo[i]
    X = train[features]
    classifier = GaussianNB()
    classifier.fit(x_train,y_train)
    Y_pred = classifier.predict(x_test)
    cm = confusion_matrix(y_test,Y_pred)
    accuracy = float(cm.diagonal().sum())/len(y_test)
    
    if accuracy > acc:
        acc = accuracy
        feat = features
        
print(acc, features)

In [ ]:
for i in range(1, 26):
    c = KNeighborsClassifier(n_neighbors = i)
    classifiers.append(c)

acc = 0
for i in range(len(combo)):
    Y = train['Survived']
    features = combo[i]
    X = train[features]
    for i in range(1, 26):
        classifier = KNeighborsClassifier(n_neighbors = i)
        classifier.fit(x_train,y_train)
        Y_pred = classifier.predict(x_test)
        cm = confusion_matrix(y_test,Y_pred)
        accuracy = float(cm.diagonal().sum())/len(y_test)
    
        if accuracy > acc:
            acc = accuracy
            feat = features
            n = i
        
print(acc, features, i)

In [18]:
for i in range(len(combo)):
    Y = train['Survived']
    features = combo[i]
    X = train[features]
    
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)
    
    model = Sequential()
    model.add(Dense(25, input_dim=2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=500, verbose=0)
        
    _, train_acc = model.evaluate(x_train, y_train, verbose=0)
    _, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(features)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
    
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='test')
    plt.legend()
    plt.show()

ValueError: in user code:

    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\whitn\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 247, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception en